<a href="https://colab.research.google.com/github/LeaslyG/proyecto_ap_refuerzo/blob/master/DQN_BQP_10000steps_Leasly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [262]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/drive'
drive_root = mount + "/My Drive/content/proyecto6"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

In [263]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

We're running Colab
Colab: mounting Google drive on  /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Colab: making sure  /content/drive/My Drive/content/proyecto6  exists.

Colab: Changing directory to  /content/drive/My Drive/content/proyecto6
/content/drive/My Drive/content/proyecto6
Archivos en el directorio: 
[]


In [264]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.5.3
  %pip install torch==2.0.1
  %pip install agents==1.4.0


  Cloning https://github.com/Kojoley/atari-py.git to /tmp/pip-req-build-cpjhng4m
  Running command git clone --filter=blob:none --quiet https://github.com/Kojoley/atari-py.git /tmp/pip-req-build-cpjhng4m
  Resolved https://github.com/Kojoley/atari-py.git to commit 86a1e05c0a95e9e6233c3a413521fdb34ca8a089
  Preparing metadata (setup.py) ... done


In [265]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [266]:
!pip install gym
import gym

In [267]:
# Creamos el entorno sobre el que vamos a ejecutar nuestro agente
env = gym.make("BreakoutDeterministic-v4")
env.seed(123)  # Definimos la semilla para la reproducibilidad de la ejecución

[123, 151010689]

In [268]:
# Dimensions de las observaciones del entorno
print("Dimensiones del espacio de observaciones: ")
print(env.observation_space.shape)

Dimensiones del espacio de observaciones: 
(210, 160, 3)


In [269]:
print("Formato de las observaciones:")
env.observation_space
env.action_space

Formato de las observaciones:


Discrete(4)

In [270]:
if IN_COLAB:
  !pip install gym[atari-py]
  !apt-get install -y xvfb ffmpeg
  !pip install imageio

  import gym
  import imageio
  import numpy as np
  import matplotlib.pyplot as plt
  from IPython.display import HTML
  from base64 import b64encode

  # Función para crear un video a partir de imágenes
  def create_video(frames, video_name):
      imageio.mimsave(video_name, frames, fps=30)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.10).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [273]:
from __future__ import division

from PIL import Image
import numpy as np
import gym
from gym.wrappers import AtariPreprocessing, FrameStack
import json
import os

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Input, concatenate, Lambda, Conv2D
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import Callback, FileLogger, ModelIntervalCheckpoint
from rl.core import Processor

In [274]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4


In [275]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channels)
        if observation.shape[2] == 3:
            observation = np.mean(observation, axis=2).astype(np.uint8)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        # Replicar la observación procesada a través de los 4 canales
        processed_observation = np.stack([processed_observation] * WINDOW_LENGTH, axis=2)
        assert processed_observation.shape == (INPUT_SHAPE[0], INPUT_SHAPE[1], WINDOW_LENGTH)
        return processed_observation

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        # Eliminar la dimensión adicional para tener la forma correcta
        if processed_batch.ndim == 5:
            processed_batch = np.squeeze(processed_batch, axis=1)
        return processed_batch

    def process_reward(self, reward):
        print("Original reward:", reward)
        processed_reward = np.clip(reward, -1., 1.)
        print("Processed reward:", processed_reward)
        return processed_reward


In [276]:
class ObservationReshaper(gym.ObservationWrapper):
    def __init__(self, env):
        super(ObservationReshaper, self).__init__(env)
        self.observation_space = gym.spaces.Box(
            low=0, high=255, shape=(INPUT_SHAPE[0], INPUT_SHAPE[1], WINDOW_LENGTH), dtype=np.uint8)

    def observation(self, obs):
        # Reorganiza la observación de (WINDOW_LENGTH, 84, 84, 1) a (84, 84, WINDOW_LENGTH)
        if obs.shape == (WINDOW_LENGTH, INPUT_SHAPE[0], INPUT_SHAPE[1], 1):
            obs = np.squeeze(obs, axis=-1)  # Elimina la dimensión redundante
            obs = np.transpose(obs, (1, 2, 0))  # Transpone para obtener la forma correcta
        return obs

In [322]:
class EpisodeRewardLogger(Callback):
    def __init__(self, rewards_filename):
        self.rewards_filename = rewards_filename
        if os.path.exists(rewards_filename):
            with open(rewards_filename, 'r') as f:
                self.rewards = json.load(f)
        else:
            self.rewards = []

    def on_episode_end(self, episode, logs={}):
        reward = logs.get('episode_reward')
        print(f"Episode: {episode}, Reward: {reward}")
        self.rewards.append(reward)
        self.save_rewards()

    def save_rewards(self):
        with open(self.rewards_filename, 'w') as f:
            json.dump(self.rewards, f)

In [323]:
def construct_q_network(num_actions,dim=(84,84,4), loss='mse', optimizer=Adam(lr=0.00001)):
    # Uses the network architecture found in DeepMind paper
    model = Sequential()
    model.add(Convolution2D(32, (8, 8), strides=(4, 4), input_shape=dim))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(num_actions))
    model.compile(loss=loss, optimizer=optimizer)
    print("Successfully constructed networks.")
    return model

In [324]:
print("Archivos en el directorio actual:")
print(os.listdir('.'))

Archivos en el directorio actual:
['gym_video.mp4', 'gym_video_episode_1.mp4', 'gym_video_episode_2.mp4', 'gym_video_episode_3.mp4', 'gym_video_episode_4.mp4', 'gym_video_episode_5.mp4', 'dqn_SpaceInvaders-v0_weights.h5f.data-00000-of-00001', 'dqn_SpaceInvaders-v0_weights.h5f.index', 'checkpoint']


In [325]:
env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)
env = AtariPreprocessing(env, frame_skip=1, grayscale_newaxis=True)
env = FrameStack(env, num_stack=WINDOW_LENGTH)
env = ObservationReshaper(env)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n
print("Numero de acciones disponibles: " + str(nb_actions))
print("Formato de las observaciones: " + str(env.observation_space))

Numero de acciones disponibles: 6
Formato de las observaciones: Box(0, 255, (84, 84, 4), uint8)


In [326]:
memory = SequentialMemory(limit=1000000, window_length=1)

In [327]:
policy = BoltzmannQPolicy()

In [328]:
model= construct_q_network(nb_actions, dim=(INPUT_SHAPE[0], INPUT_SHAPE[1], WINDOW_LENGTH), loss='mse', optimizer=Adam(lr=1e-4))
model.summary()
processor = AtariProcessor()

dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, processor=processor,
               nb_steps_warmup=10, gamma=0.9,
               target_model_update=1e-2, policy=policy)

# Compila el agente DQN
dqn.compile(Adam(learning_rate=1e-4), metrics=['mae'])

Successfully constructed networks.
Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_45 (Conv2D)          (None, 20, 20, 32)        8224      
                                                                 
 activation_60 (Activation)  (None, 20, 20, 32)        0         
                                                                 
 conv2d_46 (Conv2D)          (None, 9, 9, 64)          32832     
                                                                 
 activation_61 (Activation)  (None, 9, 9, 64)          0         
                                                                 
 conv2d_47 (Conv2D)          (None, 7, 7, 64)          36928     
                                                                 
 activation_62 (Activation)  (None, 7, 7, 64)          0         
                                                                 
 flatten_15 (Flatt

In [369]:

# Define los nombres de archivos para guardar los pesos y los registros
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(env_name)
rewards_filename = 'dqn_{}_rewards.json'.format(env_name)

# Define los callbacks, incluyendo el logger personalizado
reward_logger = EpisodeRewardLogger(rewards_filename)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=2500)]
callbacks += [FileLogger(log_filename, interval=100)]
callbacks += [reward_logger]

sample_observation = env.reset()
print(f"Sample observation shape: {np.array(sample_observation).shape}")

Sample observation shape: (84, 84, 4)


In [370]:
# Entrena el agente
print("Starting training...")
history=dqn.fit(env, callbacks=callbacks, nb_steps=10000, log_interval=1000, visualize=False)

Se han truncado las últimas 5000 líneas del flujo de salida.
 507/1000 [==============>...............] - ETA: 2:12 - reward: 0.0118Original reward: 0.0
Processed reward: 0.0
 508/1000 [==============>...............] - ETA: 2:12 - reward: 0.0118Original reward: 0.0
Processed reward: 0.0
 509/1000 [==============>...............] - ETA: 2:12 - reward: 0.0118Original reward: 0.0
Processed reward: 0.0
 510/1000 [==============>...............] - ETA: 2:11 - reward: 0.0118Original reward: 0.0
Processed reward: 0.0
 511/1000 [==============>...............] - ETA: 2:11 - reward: 0.0117Original reward: 0.0
Processed reward: 0.0
 512/1000 [==============>...............] - ETA: 2:11 - reward: 0.0117Original reward: 0.0
Processed reward: 0.0
 513/1000 [==============>...............] - ETA: 2:11 - reward: 0.0117Original reward: 0.0
Processed reward: 0.0
 514/1000 [==============>...............] - ETA: 2:11 - reward: 0.0117Original reward: 0.0
Processed reward: 0.0
 515/1000 [==============>.

In [371]:
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
dqn.save_weights(weights_filename, overwrite=True)
print(f"Pesos guardados en {weights_filename}")


Pesos guardados en dqn_SpaceInvaders-v0_weights.h5f


In [372]:
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Se han truncado las últimas 5000 líneas del flujo de salida.
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed rewar

In [373]:
processed_rewards = history.history['episode_reward']
mean_processed_reward = np.mean(processed_rewards)
print("Mean processed reward:", mean_processed_reward)

Mean processed reward: 9.133333333333333


In [377]:
import json
import numpy as np
log_filename = 'dqn_{}_log.json'.format(env_name)

with open(log_filename, 'r') as f:
    log_data = json.load(f)
episode_rewards = log_data.get('episode_reward', [])
print("Processed rewards:", episode_rewards)

if episode_rewards:
    max_processed_reward = np.max(episode_rewards)
    print("Max processed reward:", max_processed_reward)
else:
    print("No rewards to display.")

Processed rewards: [9.0, 5.0, 9.0, 6.0, 13.0, 11.0, 5.0, 7.0, 11.0, 21.0, 8.0, 6.0, 4.0, 4.0, 18.0]
Max processed reward: 21.0
